In [2]:
import requests
import pandas as pd
from config import app_id, api_key

In [3]:
df = pd.read_csv('austin_locations_all.csv')
df.head()

Unnamed: 0               name                  brand_id  \
0          43             Subway  513fbc1283aa2dc80c000005   
1          44      Auntie Anne's  513fbc1283aa2dc80c00013e   
2          45  Thundercloud Subs  513fbc1283aa2dc80c000b53   
3          46       Schlotzsky's  513fbc1283aa2dc80c00013a   
4          47      Auntie Anne's  513fbc1283aa2dc80c00013e   

                      fs_id                         address  address2    city  \
0                       NaN              6422 HIGHWAY 183 S       NaN  Austin   
1  4cadcc8bd1f8b60c8cb771c6  3600 Presidential Blvd Ste 204       NaN  Austin   
2                       NaN     3600 PRESIDENTIAL BLVD #213       NaN  Austin   
3                       NaN          3600 PRESIDENTIAL BLVD       NaN  Austin   
4                       NaN  3600 PRESIDENTIAL BLVD STE 204       NaN  Austin   

  state country    zip         phone                     website  \
0    TX      US  78744  1.512243e+10       http://www.subway.com   
1    TX      US  78719           NaN  http://www.auntieannes.com   
2    TX      US  78719  1.512530e+10    http://thundercloud.com/   
3    TX      US  78719  1.512530e+10  http://www.schlotzskys.com   
4    TX      US  78719  1.512385e+10  http://www.auntieannes.com   

                                               guide       id        lat  \
0  <p><b id="docs-internal-guid-6dd52ca7-83bb-38f...   796835  30.165001   
1                                                NaN  2492829  30.202162   
2                                                NaN   790078  30.202148   
3                                                NaN  1869663  30.202635   
4                                                NaN   842312  30.202999   

         lng                created_at                updated_at  distance_km  
0 -97.693001  2017-06-26T21:54:34.000Z  2017-08-20T14:07:28.000Z    33.636331  
1 -97.666260  2018-09-21T14:06:53.000Z  2019-01-05T14:56:54.000Z    34.079371  
2 -97.666832  2017-06-26T21:53:40.000Z  2017-08-20T14:06:12.000Z    34.119790  
3 -97.667381  2017-07-12T17:55:52.000Z  2017-08-20T14:06:32.000Z    34.195123  
4 -97.667999  2017-06-26T21:59:43.000Z  2017-08-20T14:06:34.000Z    34.266501

In [4]:
brands = df['brand_id'].unique()
len(brands)

121

In [5]:
menu_url = 'https://trackapi.nutritionix.com/v2/search/instant'
headers = {'x-app-id': app_id, 'x-app-key':api_key}

In [32]:
branded_item_list = []
for brand in brands:
    params = {'query': 'burrito', 'brand_ids': brand, 'branded': True, 'branded_type': 1, 'detailed': True}
    response_menu_items = requests.get(menu_url, headers=headers, params=params).json()
    if response_menu_items['branded'] != []:
        branded_item_list.append(response_menu_items['branded'])

In [33]:
item_brand = [branded_item_list[i][j]['brand_name'] for i in range(len(branded_item_list)) for j in range(len(branded_item_list[i]))]
item_name = [branded_item_list[i][j]['food_name'] for i in range(len(branded_item_list)) for j in range(len(branded_item_list[i]))]
nutrient_dict = [branded_item_list[i][j]['full_nutrients'] for i in range(len(branded_item_list)) for j in range(len(branded_item_list[i]))]
calories = [branded_item_list[i][j]['nf_calories'] for i in range(len(branded_item_list)) for j in range(len(branded_item_list[i]))]

In [34]:
items_df = pd.DataFrame({'Restaurant': item_brand,
                        'Food_Name': item_name,
                        'Nutrients': nutrient_dict,
                        'Calories': calories})
items_df.tail(10)

Restaurant                                          Food_Name  \
302  BJ's Restaurants  Best Beginnings Appetizer Combo With Cherry Ch...   
303  McAlister's Deli                             Sweet Chipotle Chicken   
304  McAlister's Deli                      Chipotle Chicken & Bacon Spud   
305  McAlister's Deli          Sweet Chipotle Chicken (Choose 2 Portion)   
306  McAlister's Deli   Chipotle Chicken & Bacon Spud (Choose 2 Portion)   
307     Boston Market                            Chipotle Ranch Dressing   
308       Smashburger                             Chipotle Mayo Dressing   
309       Smashburger                                 Spicy Chipotle Bun   
310       Smashburger                               Spicy Chipotle Sauce   
311       Smashburger                       Side of Spicy Chipotle Sauce   

                                             Nutrients  Calories  
302  [{'value': 84, 'attr_id': 203}, {'value': 120,...      2280  
303  [{'value': 48, 'attr_id': 203}, {'value': 15, ...       630  
304  [{'value': 62, 'attr_id': 203}, {'value': 46, ...      1200  
305  [{'value': 24, 'attr_id': 203}, {'value': 7, '...       310  
306  [{'value': 31, 'attr_id': 203}, {'value': 23, ...       600  
307  [{'value': 2, 'attr_id': 203}, {'value': 29, '...       290  
308  [{'value': 0, 'attr_id': 203}, {'value': 56, '...       530  
309  [{'value': 5, 'attr_id': 203}, {'value': 6, 'a...       170  
310  [{'value': 0, 'attr_id': 203}, {'value': 6, 'a...        50  
311  [{'value': 0, 'attr_id': 203}, {'value': 28, '...       260

In [13]:
items_df.to_csv('burritos.csv')